In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

In [2]:
nomeFruta = "manga"#input("Fruta? ")

In [3]:
with open("Calibracao_hsv_" + nomeFruta + ".txt") as hsv:
    
    parametros = hsv.readlines()
          
    hMin = int(parametros[0])
    sMin = int(parametros[1])
    vMin = int(parametros[2])
        
    hMax = int(parametros[3])
    sMax = int(parametros[4])
    vMax = int(parametros[5])

arquivo com as coordenadas de segmentação do pedunculo
*
*
*
*
*
*
*



# Prevê área do pedúnculo

In [4]:
def preveArea(xt, yt, xb, yb):
    
    #Regiao de interesse

    Lmax = abs(xt - xb)
    Hmax = abs(yt - yb)

    RoiL = 0.5 * Lmax
    RoiH = 0.5 * Hmax

    #Esses coordenadas são as que vão delimitar a area onde a imagem vai ser cortada

    centro = (xb + xt) / 2

    centroCaixa = RoiL / 2

    x1 = centro - centroCaixa
    x2 = centro + centroCaixa

    y1 = abs(yt - 5)
    y2 = abs(RoiH - y1)

    x1 = int(x1)
    y1 = int(y1)
    x2 = int(x2)
    y2 = int(y2)
    
    return x1, y1, x2, y2   

In [5]:
def segmentacaoHSV(imgCorte):
    
    baixo = np.array([2, 67, 149])
    alto =  np.array([27, 131, 228])

    img_hsv = cv2.cvtColor(imgCorte, cv2.COLOR_BGR2HSV)

    mask = cv2.inRange(img_hsv, baixo, alto)

    img_seg = cv2.bitwise_and(imgCorte, imgCorte, mask = mask)
    
    return img_seg

In [6]:

def segmentacaoBGR(img):

    corProcuradaBlueMin  = 110
    corProcuradaGreenMin = 140
    corProcuradaRedMin   = 160

    corProcuradaBlueMax  = 140
    corProcuradaGreenMax = 180
    corProcuradaRedMax   = 230

    altura, largura, canal = img.shape

    for x in range(0, largura):

        for y in range(0, altura):

            b,g,r = img[y, x]

            if(b >= corProcuradaBlueMin  and b <= corProcuradaBlueMax   and 
               g >= corProcuradaGreenMin and g <= corProcuradaGreenMax  and 
               r >= corProcuradaRedMin   and r <= corProcuradaRedMax):

                #img[y, x] = [b,g,r]
                
                a = 1

            else:

                #img[y,x] = 0
                
                b = 0
                
    plt.imshow(img)
    plt.show
                
    if(a == 1):
        return 1
    else:
        return 0
        

In [7]:
def cantosEncontrados(imgSegBGR):
    
    gray = cv2.cvtColor(imgSegBGR, cv2.COLOR_BGR2GRAY)
    
    largura = imgSegBGR.shape[0]
    altura  = imgSegBGR.shape[1]
    
    centroX = round(largura / 2)
    centroY = round(altura / 2)

    todas_coordenadas  =  [] #Guarda todas as coordenadas (eixo X e Y)
    todas_coordenadasX =  [] #Guarda as coordenadas do eixo X vindas dos cantos encontrados
    todas_coordenadasY =  [] #Guarda as coordenadas do eixo Y vindas dos cantos encontrados

    try:
        
        cornersPonto = cv2.goodFeaturesToTrack(gray, 20, 0.2, 0.25) #Função responsavel por encontrar os cantos Harrys

        for item in cornersPonto:

            x,y = item[0]

            todas_coordenadas.append((x,y))
            todas_coordenadasX.append(int(x))
            todas_coordenadasY.append(int(y))
            #cv2.circle(imgSegBGR, (int(x),int(y)), 1, (0,0,255))

        return todas_coordenadas,todas_coordenadasX, todas_coordenadasY
    
    except TypeError:
        
        todas_coordenadas,todas_coordenadasX, todas_coordenadasY = [(centroX, centroY)], [(centroX)], [(centroY)]
        
        return todas_coordenadas,todas_coordenadasX, todas_coordenadasY
        

# Funções para encontrar o ponto de partida

## Média com peso

In [8]:
#Funções

#Calcula o peso das coordenadas

def calculaPesoY(a):
    
    calPeso = (a[0] * a[0]) + (a[1] * a[1])
    
    calPeso = np.sqrt(calPeso)
    
    calPeso = int(calPeso)
    
    return calPeso

def calculaPesoX(a, centro):
    
    if(a[0] > centro):
        
        dif = (centro - a[0])
        
        soma = a[0] + dif
        
    else:
        
        soma = a[0]
        
    calPeso = (soma * soma) + (a[1] * a[1])
    
    calPeso = np.sqrt(calPeso)
    
    calPeso = round(calPeso)
    
    return calPeso



#Faz a combinação entre a coordenadas e os pesos

def combinacao(peso: list, todas_coordenadasN: list) -> list:
    
    assert len(peso) == len(todas_coordenadasN)
 
    n = len(peso)
    
    combi = []
 
    for i in range(n):
        
        combi.append((peso[i], todas_coordenadasN[i]))
         
    return combi

#Calcula a media dos pontos

def calculaMedia(m):
    
    calMedia = (m[0] *  m[1])
    
    calMedia = int(calMedia)
    
    return calMedia

In [9]:
def encontrarPontos(img, todasCoordenadas, coordenadasX, coordenadasY):
    
    pesoY =   []
    pesoX =   []
    mediaX =  []
    mediaY =  []
    
    centro =  img.shape[0] / 2
    centro = int(centro)
     
    #Calcula o peso de Y
    for i in todasCoordenadas:

        aux = calculaPesoY(i)

        pesoY.append(aux)

    #Calcula o peso de X
    for i in todasCoordenadas:

        aux = calculaPesoX(i, centro)

        pesoX.append(aux)

    #Somatorio do peso Y
    somatorioPesoY = sum(pesoY)

    #Somatoria do pedo X
    somatorioPesoX = sum(pesoX)

    #Realiza combinação
    combinacaoX = combinacao(pesoX, coordenadasX)
    combinacaoY = combinacao(pesoY, coordenadasY)

    #Calcula a media dos pontos X
    for i in combinacaoX:

        aux = calculaMedia(i)

        mediaX.append(aux)

    #Calcula a media dos pontos Y
    for i in combinacaoY:

        aux = calculaMedia(i)

        mediaY.append(aux)


    somatorioX =  sum(mediaX)
    somatorioY =  sum(mediaY)

    pontoX = (somatorioX / somatorioPesoX)  
    pontoY = (somatorioY / somatorioPesoY)

    pontoX = int(pontoX)
    pontoY = int(pontoY)

    return pontoX, pontoY

## Cálculo da diferença de cores 

In [10]:
def difEuclidiana(cores):
    
    euclidiana = (cores[0] - cores[1])
    
    Eucli = euclidiana * euclidiana
    
    return Eucli

In [11]:
def calculaEuclidiana(combinacoes):
    
    Euclidiana = 0
    
    for i in combinacoes:
        
        Euclidiana = Euclidiana + difEuclidiana(i)

        raizEuclidiana = np.sqrt(Euclidiana)
        
    return raizEuclidiana 

# Encontra ponto final

In [12]:
def marca_ponto(img, ponto_y, ponto_x):
    
    minimo = [45, 45, 45]
    maximo = [180, 180, 180]
    
    #minimo = [61, 167, 155]
   # maximo = [71, 177, 165]
    
    controle = 0
    
    #Verifica se os valores de um determinado pixel é maior que o RGB(100, 120, 40)
    
    if np.any(img[int(ponto_y), int(ponto_x)] >= minimo):
        
        if np.any(img[int(ponto_y), int(ponto_x)] <= maximo):
             
            controle = 1        
    else:
        
        controle = 0
        
    return controle

In [13]:
def localizaPonto(img, pontoY, pontoX, corPedun):
    
    copiaX, copiaY = pontoX, pontoY
    
    largura = img.shape[0]
    altura  = img.shape[1]
        
    larguraCentro = largura / 2
    alturaCentro  = altura /  2
    
    limiteBusca = larguraCentro
    
    #montar quadrante
    
    if(pontoX <= larguraCentro and pontoY <= alturaCentro):
        
        movimentoX, movimentoY = 1, 1
        
    elif(pontoX > larguraCentro and pontoY < alturaCentro):
        
        movimentoX, movimentoY = -1, 1
        
    elif(pontoX < larguraCentro and pontoY > alturaCentro):
        
        movimentoX, movimentoY = 1, 1
        
    else:
        
        movimentoX, movimentoY = -1, 1
    
    qtdBusca = 0
    
    while(1):
        
        try:
    
            coordenada = marca_ponto(img, round(pontoY), round(pontoX))

            if(coordenada == 1):

                pontoY, pontoX = pontoY, pontoX
                
                combinacaoCores = combinacao(corPedun, img[pontoY, pontoX])
                calEuclidiana = calculaEuclidiana(combinacaoCores)

                return int(pontoY), int(pontoX), calEuclidiana 

            elif(qtdBusca >= limiteBusca):
                
                combinacaoCores = combinacao(corPedun, img[int(copiaY), int(copiaX)])
                calEuclidiana = calculaEuclidiana(combinacaoCores)

                return int(copiaY), int(copiaX), calEuclidiana

            elif(pontoX == largura - 5 or pontoY == altura - 5):

                pontoX, pontoY = copiaX, copiaY

            else:

                pontoX, pontoY = pontoX + movimentoX, pontoY + movimentoY

            qtdBusca = qtdBusca + 1

            
        except IndexError:
            
            combinacaoCores = combinacao(corPedun, img[int(copiaY), int(copiaX)])
            calEuclidiana = calculaEuclidiana(combinacaoCores)
            
            return int(copiaY), int(copiaX), calEuclidiana

In [14]:
def deslocamentoCima(img, pontoY, pontoX, corPedun):
    
    #LEMBRAR QUE O OPENCV TRABALHA COM O EIXO 0 Y NA PARTE SUPERIOR ESQUERDA
    
    copiaX, copiaY = pontoX, pontoY

    limiteBusca = pontoY
    
    pontoY = pontoY - 3 #Isso faz com que o ponto seja deslocado 3 pixel para cima
    
    qtdBusca = 0
    
    while(1):
        
        try:
            
            coordenada = marca_ponto(img, round(pontoY), round(pontoX))

            if(coordenada == 1):

                pontoY, pontoX = pontoY, pontoX
                
                combinacaoCores = combinacao(corPedun, img[int(pontoY), int(pontoX)])
                calEuclidiana = calculaEuclidiana(combinacaoCores)

                return int(pontoY), int(pontoX), calEuclidiana              
 
            elif(qtdBusca >= limiteBusca):
        
                combinacaoCores = combinacao(corPedun, img[int(copiaY), int(copiaX)])
                calEuclidiana = calculaEuclidiana(combinacaoCores)
                
                return int(copiaY), int(copiaX), calEuclidiana
            
            else:
                
                pontoY, pontoX = pontoY - 1, pontoX
                
            qtdBusca = qtdBusca + 1
                
        except IndexError:
            
            combinacaoCores = combinacao(corPedun, img[int(copiaY - 3), int(copiaX)])
            calEuclidiana = calculaEuclidiana(combinacaoCores)
                        
            return  int(copiaY), int(copiaX), calEuclidiana

In [15]:
def deslocamentoBaixo(img, pontoY, pontoX, corPedun):
    
    #LEMBRAR QUE O OPENCV TRABALHA COM O EIXO 0 Y NA PARTE SUPERIOR ESQUERDA
    
    copiaX, copiaY = pontoX, pontoY
    
    altura  = img.shape[1]
    
    limiteBusca = altura - pontoY
    
    pontoY = pontoY + 3 #Isso faz com que o ponto seja deslocado 3 pixel para baixo
    
    qtdBusca = 0
    
    while(1):
        
        try:
            
            coordenada = marca_ponto(img, round(pontoY), round(pontoX))

            if(coordenada == 1):

                pontoY, pontoX = pontoY, pontoX
                
                combinacaoCores = combinacao(corPedun, img[int(pontoY), int(pontoX)])
                calEuclidiana = calculaEuclidiana(combinacaoCores)

                return int(pontoY), int(pontoX), calEuclidiana 

            elif(qtdBusca >= limiteBusca):
                
                combinacaoCores = combinacao(corPedun, img[int(copiaY), int(copiaX)])
                calEuclidiana = calculaEuclidiana(combinacaoCores)

                return int(copiaY), int(copiaX), calEuclidiana
            
            else:
                
                pontoY, pontoX = pontoY + 1, pontoX
                
            qtdBusca = qtdBusca + 1
                
        except IndexError:
            
            combinacaoCores = combinacao(corPedun, img[int(copiaY + 3), int(copiaX)])
            calEuclidiana = calculaEuclidiana(combinacaoCores)
            
            return  int(copiaY), int(copiaX), calEuclidiana

In [16]:
def encontraPontoFinal(origemX, origemY, origemCal, 
                       cimaX, cimaY, cimaCal, 
                       baixoX, baixoY, baixoCal):
    
    if(origemCal < cimaCal and origemCal < baixoCal):
        
        return origemX, origemY, origemCal
    
    elif(cimaCal < origemCal and cimaCal < baixoCal):
        
        return cimaX, cimaY, cimaCal
    
    else:
        
        return baixoX, baixoY, baixoCal

# Identificação

In [22]:
#cap = cv2.VideoCapture(1)

while True:

    #ret, frame = cap.read()
    
    frame = cv2.imread("normal111.jpg")
    hsv =  cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    imgPedunculo = np.copy(frame)

    lim_inf = np.array([hMin, sMin, vMin])
    lim_sup = np.array([hMax, sMax, vMax])
        
    color_mask = cv2.inRange(hsv, lim_inf, lim_sup)

    (couts,hir) = cv2.findContours(color_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    idx = 0
    info_coordenadas = []
  
    for cout in couts:
        
        area = cv2.contourArea(cout)

        if (area > 600):
            
            idx = idx + 1

            x,y,w,h = cv2.boundingRect(cout)
            frame = cv2.rectangle(frame, (x,y),(x+w,y+h),(0,0,255),2)
            
            xt = x
            xb = x+w
            yt = y
            yb = y+h

            corPedunculo = [66, 172, 160] #Cor do pedunculo encontrado no histograma
            
            area = preveArea(xt, yt, xb, yb) #Função que preve a area
            
            x1, y1, x2, y2 = area[0], area[1], area[2], area[3] #Pega as coordenadas
            
            TopLeftX = x1
            TopLeftY = y2            

            corte = imgPedunculo[y2:y1, x1:x2] #Faz o corte na area prevista
            
            cv2.putText(frame, (str(idx)), (x1, y2), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255))
            
            cv2.imwrite(str(idx) + ".jpg", corte) #Salva as imagens cortadas
                   
            cv2.rectangle(frame, (x2, y2), (x1,y1), (255,255,0), 1) #Desenha um retangulo na area encontrada
            
            corteHSV = segmentacaoHSV(corte) #Realiza a segmentacao HSV na area do pedunculo - é preciso fazer
                                             #a calibracao no outro algoritmo
                      
            cantos = cantosEncontrados(corteHSV) #Encontra cantos na imagem
            
            todas, x, y = cantos[0], cantos[1], cantos[2] #Pega todos os cantos como:
                                                                                     #todos (x,y)
                                                                                    #Somente (x)
                                                                                   #Somente (y)
            
            pontos = encontrarPontos(corteHSV, todas, x, y) #Encontra o ponto de partida:
                                                                                    #imagem
                                                                                   #todas as coordenadas
                                                                                  #coordenadas x e y
            
            pontoPartidaX, pontoPartidaY = pontos[1], pontos[0] #Pega a coordenada do ponto de partida (x, y)
            cv2.circle(corteHSV, (pontoPartidaY, pontoPartidaX), 1, (255, 0, 255)) #purple
                                             
            #Primeira coordenada encontrada, e sua distancia euclidiana
            pontosP = localizaPonto(corteHSV, pontoPartidaX, pontoPartidaY, corPedunculo) 
            pontoP1, pontoP2, calculoEuclidianaP = pontosP[0], pontosP[1], pontosP[2] 
            cv2.circle(corteHSV, (pontoP2, pontoP1), 1, (0, 0, 255))#red
            
            #Segunda coordenada encontrada, movimento para cima, e sua distancia euclidiana
            deslocamentoCimaT = deslocamentoCima(corteHSV, pontoP1, pontoP2, corPedunculo)
            cimaY, cimaX, calCima = deslocamentoCimaT[0], deslocamentoCimaT[1], deslocamentoCimaT[2]
            cv2.circle(corteHSV, (cimaX, cimaY), 1, (0,255,0))#green
            
            #Terceira coordenada encontrada, movimento para baixo, e sua distancia euclidiana
            deslocamentoBaixoT = deslocamentoBaixo(corteHSV, pontoP1, pontoP2, corPedunculo)
            baixoY, baixoX, calBaixo = deslocamentoBaixoT[0], deslocamentoBaixoT[1], deslocamentoBaixoT[2]
            cv2.circle(corteHSV, (baixoX, baixoY), 1, (255, 255, 255)) #white
            
            #Calcula qual ponto tem a menor distancia
            pontoFinal = encontraPontoFinal(pontoP1, pontoP2, calculoEuclidianaP, 
                                            cimaY, cimaX, calCima, 
                                            baixoY, baixoX, calBaixo)
            
            #Seleciona o que tem a menor distancia euclidiana
            pontoFinalX, pontoFinalY, euclidiana = pontoFinal[0], pontoFinal[1], pontoFinal[2]
            
            cv2.circle(corteHSV, (pontoFinalY, pontoFinalX), 2, (218,165,32))
            
            cv2.imshow(str(idx) + "ROI HSV", corteHSV)
            
            pontoFinalX = abs(pontoFinalX)
            pontoFinalY = abs(pontoFinalY)
            
            #Encontra a coordenada final na imagem completa
            CDFX = TopLeftX + pontoFinalY
            CDFY = TopLeftY + pontoFinalX
            
            cv2.circle(frame, (CDFX, CDFY), 2, (0, 0, 255))
            #cv2.circle(frame, (CDFX, TopLeftY), 2, (0, 0, 255))
            #cv2.circle(frame, (TopLeftX, CDFY), 2, (255, 0, 255))
            
            TopLeftX = 0
            TopLeftY = 0
            
            info_coordenadas.append((idx, CDFX, CDFY, euclidiana))

    cv2.imshow('Identificacao por HSV', frame) 
   
    if cv2.waitKey(1) == 27:
        break

cv2.destroyAllWindows()
#cap.release()    

In [23]:
info_coordenadas

[(1, 61, 288, 57.23635208501674),
 (2, 220, 191, 69.13031173081747),
 (3, 148, 172, 55.937465083787984),
 (4, 14, 173, 51.40038910358559),
 (5, 184, 82, 42.68489194082609),
 (6, 102, 44, 63.474404290233394)]